Resources

https://drive.google.com/drive/folders/1VD8J-zYix3mD31jbIBhqK2fke-

https://github.com/nlpcuom/WEIntrinsicEvaluation

## Obectives

1. Experiment CBOW and skip gram model with LOgistic regression
2. Train own wor2vec
3. Fine tuning word2vec 

## Word2vec model with logistic regression

size: The number of dimensions of the embeddings and the default is 100.

window: The maximum distance between a target word and words around the target word. The default window is 5.

min_count: The minimum count of words to consider when training the model; words with occurrence less than this count will be ignored. The default for min_count is 5.

workers: The number of partitions during training and the default workers is 3.

sg: The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW

## Load modules and datasets

In [ ]:
import neptune
from neptunecontrib.monitoring.metrics import expand_prediction, log_class_metrics, log_binary_classification_metrics, log_classification_report,log_confusion_matrix,log_prediction_distribution
from neptunecontrib.api import log_table
import os
from dotenv import load_dotenv

load_dotenv()
NEPTUNE_PROJECT= os.getenv('NEPTUNE_PROJECT')
NEPTUNE_API_TOKEN = os.getenv(('NEPTUNE_API_TOKEN'))
neptune.init(project_qualified_name= NEPTUNE_PROJECT,api_token=NEPTUNE_API_TOKEN) 
             

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
#from sklearn.metrics import accuracy_score, f1_score, precision_score,roc_curve,roc_auc_score,confusion_matrix,recall_score
from sklearn.pipeline import Pipeline
import re
import gensim

# import helper function script
import sys
sys.path.insert(1,'G:\\Github\\Sinhala-Hate-Speech-Detection')
import utills

In [2]:
# load datasets
path = '../Datasets/processed/no_preprocessing/'
df_A = pd.read_csv(path+'df_A.csv')    
df_B = pd.read_csv(path+'df_B.csv')    # fb dataset -kaggle 
df_A_B = pd.read_csv(path+'df_A_B.csv') 


## DBOW - Distributed bag of word

In [ ]:
from gensim.models.doc2vec import TaggedDocument
train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['narrative']), tags=[r.Product]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['narrative']), tags=[r.Product]), axis=1)
train_tagged.values[30]


In [ ]:
import multiprocessing

cores = multiprocessing.cpu_count()

In [ ]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)]

In [ ]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressorsdef vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [ ]:
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

## Distributed memory

In [ ]:
## 
model_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=300, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065)
model_dmm.build_vocab([x for x in tqdm(train_tagged.values)])

In [ ]:
%%time
for epoch in range(30):
    model_dmm.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dmm.alpha -= 0.002
    model_dmm.min_alpha = model_dmm.alpha

In [ ]:
y_train, X_train = vec_for_learning(model_dmm, train_tagged)
y_test, X_test = vec_for_learning(model_dmm, test_tagged)

logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

In [ ]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
new_model = ConcatenatedDoc2Vec([model_dbow, model_dmm])

In [ ]:
def get_vectors(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [ ]:
y_train, X_train = get_vectors(new_model, train_tagged)
y_test, X_test = get_vectors(new_model, test_tagged)

logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

In [6]:
print(model.wv.most_similar('ලැජ්ජයි'))
print(model.wv.most_similar('මෑරෙන්න'))
print(model.wv.most_similar('යකො'))
print(model.wv.most_similar('හොරා'))
print(model.wv.most_similar('බල්ලා'))
print(model.wv.most_similar('තම්බි'))
print(model.wv.most_similar('හුත්ති'))

[('ෂිකේ.', 0.624288022518158), ('දැක්කහමමචං', 0.606025218963623), ('අසරනට', 0.6009397506713867), ('ගානවාවල', 0.5993783473968506), ('තිලකයියේ', 0.5935572385787964), ('නාගෙනදඟලන්න', 0.5622760057449341), ('හිත්න්නත්', 0.5605373978614807), ('ආණ්දුව', 0.5562554597854614), ('්ගමයාවිලි', 0.5535441637039185), ('දැක්කම\tපිනා', 0.5499239563941956)]
[('හෑබෑමයි', 0.6853367686271667), ('වෙන්නජීවත්', 0.615235447883606), ('වෙලාවකඉපදුනේ', 0.5789225101470947), ('හෙටවෙයිද', 0.5657752752304077), ('ලෑස්තිවෙයල්ල', 0.5579881072044373), ('\u200dබොම්බ', 0.5476895570755005), ('දෙයක්අන්ඩුවට', 0.5422118306159973), ('දෑක්කනම්', 0.5235146880149841), ('ෆොන්නඩෝටත්', 0.5233187675476074), ('වෑස්සටවත්', 0.5230247974395752)]
[('යකෝ', 0.6215049624443054), ('පුරුන්ග', 0.5947710871696472), ('හොයපි', 0.5878672003746033), ('වේසික', 0.5774468779563904), ('සඳරුවණ්', 0.5767057538032532), ('බන්', 0.5762976408004761), ('පෑන්චියො', 0.5738309621810913), ('මේරටෙ', 0.5726816058158875), ('තිරිසනාතොපෙ', 0.572677731513977), ('දුශ්ඨයෙක්ට

## LR

In [1]:
X_train, X_test, Y_train, Y_test = utills.prepare_dataset(df_A, "df_A")

NameError: name 'utills' is not defined

In [ ]:
def document_vector(model,doc):
    """Create document vectors by averaging word vectors. Remove out-of-vocabulary words."""
    doc = [word for word in doc if word in model.wv.vocab]
    return np.mean(w2v[doc], axis=0)

In [ ]:
X_train_avg_vec = [document_vector(model, doc) for doc in X_train ]#data.ingredients.apply(document_vector)
X_test_avg_vec = [document_vector(model, doc) for doc in X_test ]    # test.ingredients.apply(document_vector)

In [ ]:
X = list(data['doc_vector'])
X_test = list(test['doc_vector'])

In [ ]:
clf = LogisticRegression(C=100)
clf.fit(X_train_avg_vec, Y_train)

In [ ]:
y_pred = clf.predict(X_test_avg_vec)
utills.result(Y_test, y_pred)

In [ ]:
# The train and test data was converted to list of strings since word2vec requires that
#train_corpus = [" ".join() for x in X_train]
train_corpus = [x.split() for x in X_train]
test_corpus = [x.split() for x in X_test]  

In [ ]:
import gensim
# let X be a list of tokenized texts (i.e. list of lists of tokens)
# skip gram model - good for high morphological languafes
model = gensim.models.Word2Vec(train_corpus, vector_size=300,window = 10,min_count=1,sg=1)
print(model.wv)
print(len(model.wv.vectors))
w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))


In [ ]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec,size = 300):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = size #len(word2vec.itervalues().next())

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


In [ ]:
unique, frequency = np.unique(Y_pred_w2v,  return_counts = True)
print(unique,frequency)

In [ ]:
lr = LogisticRegression()
w2v_lr_pipe = Pipeline([('w2v', MeanEmbeddingVectorizer(w2v)), ('lr', lr)])


w2v_lr_pipe.fit(X_train, Y_train)

Y_pred_w2v = w2v_lr_pipe.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(Y_test, Y_pred_w2v))

utills.confusion_Matrix(Y_test,Y_pred_w2v)

utills.PlotRocAuc(Y_test, Y_pred_w2v, "green", "w2v_clf")

## Use pretrained word2vec CBOW

## Use pretrained word2vec Skip gram

## fine tuned pretrained one

In [ ]:
new_model.build_vocab(df_A.comment.values, update = True)
new_model.train(df_A.comment.values, total_examples=2, epochs = 1)
new_model.wv.vocab

## Train your own word2vec

## Test with logistic regression

### Helper functions

In [ ]:
def classifier_feature(datasets,models,features):
    final_result =pd.DataFrame(columns=['Accuracy','F1-score','Recall','Precision','AUC'])
    for df_name,df in datasets.items():
        df_result =pd.DataFrame(columns=['Accuracy','F1-score','Recall','Precision','AUC'])
        X_train,X_test,Y_train,Y_test = utills.prepare_dataset(df,df_name)
        for feature_name,feature in features.items():
            feature_result =pd.DataFrame(columns=['Accuracy','F1-score','Recall','Precision','AUC'])
            X_train_f,X_test_f = feature(X_train,X_test)
            for model_name,model in models.items():
                name = df_name + "+" + feature_name+ "+"+ model_name
                print(name)
                Y_pred = model(X_train_f,X_test_f,Y_train)
                accuracy, f1_score, recall, precision, auc = result(Y_test,Y_pred)
                final_result.loc[name] = [accuracy, f1_score, recall, precision, auc]
                feature_result.loc[model_name] =[accuracy, f1_score, recall, precision, auc]
                key =model_name + "+"+ feature_name
                df_result.loc[key] = [accuracy, f1_score, recall, precision, auc]
                log_result(Y_test,Y_pred,name,df_name,feature_name,model_name)
            print(" ==== ",feature_name ," ==== ")
            display(feature_result)
            log_table(feature_name,feature_result)
        print(" ==== ",df_name ," ==== ")
        display(df_result)
        log_table(df_name,df_result)
    display(final_result)
    log_table(name,final_result)



In [ ]:
def bow_word(X_train,X_test):
    bow = CountVectorizer(analyzer="word", tokenizer=lambda text: text.split(),ngram_range=(1,2),lowercase=False)
    bow.fit(X_train)
    X_train_bow = bow.transform(X_train)
    X_test_bow = bow.transform(X_test)

    #print(bow.get_feature_names()[:20])
    #print('The shape is', bow.shape)
    # postion
    #print(bow.vocabulary_)

    return X_train_bow,X_test_bow

In [ ]:
def lr(X_train,X_test,Y_train):
    lr = LogisticRegression(C=10,max_iter=350)
    lr.fit(X_train,Y_train)
    Y_pred = lr.predict(X_test)
    return Y_pred

In [ ]:
df_dict = {'df_A': df_A,'df_B': df_B,'df_C' :df_C,'df_D' :df_D,'df_A_B' :df_A_B,'df_A_C_D':df_A_C_D,'df_A_B_C':df_A_B_C,'df_A_B_D':df_A_B_D,'df_B_C_D':df_B_C_D,'df_all':df_all}
# feature dict
feature_dict={"bow_word":bow_word,"bow_char":bow_char,"tfidf_word":tfidf_word,"tfidf_char":tfidf_char}
# model dict
model_dict = {"LR":lr,"SVC":svc,"MNB":MNB}
classifier_feature(df_dict, model_dict, feature_dict)